<a id='top'></a>
# NIRSpec IFU pipeline processing

**Author**: James Muzerolle

Plotting function originally developed by Bryan Hilbert

**Latest Update**: June 10, 2022

## Table of Contents
* [Introduction](#intro)
    * [Overview](#overview)
    * [Simulated data](#sims)
* [Imports](#imports)
* [Convenience functions](#func)
* [Pipeline processing flag](#flag)
* [Download data](#data)
* [Input data](#inputs)
* [Level 2 association files](#lvl2asn)
* [Run the calwebb_spec2 pipeline](#runspec2)
* [Run individual steps of the spec2 pipeline](#runspec2steps)
    * [assign_wcs](#awcs)
    * [background](#background)
    * [imprint](#imprint)
    * [sourcetype](#srctype)
    * [flat_field](#flat)
    * [pathloss](#pathloss)
    * [photom](#photom)
    * [cube_build](#cubebuild)
    * [extract_1d](#extract1d)
* [Run the calwebb_spec3 pipeline](#runspec3)

## Introduction <a id='intro'></a>

### Overview <a id='overview'></a>

In this notebook, we will explore the stage 2 and 3 pipelines for NIRSpec IFU data. Here, we will focus on the mechanics of processing "real" example data, including how to use associations for background subtraction and multi-exposure combination, what particular steps actually do to the data, and what the primary data products at each stage look like. We will also briefly examine how to interact and work with data models for each product.

We are using pipeline version 1.1.0 for all data processing in this notebook. Most of the processing runs shown here use the default reference files from the Calibration Reference Data System (CRDS), with one exception at the end to show an example of how to modify/override. Please note that pipeline software development is a continuous process, so results in some cases may be slightly different if using a subsequent version. There are also a few known issues with some of the pipeline steps in this build that are expected to be fixed in the near future, though these do not significantly effect the products you will see here. Finally, some of the calibration reference files used by individual pipeline steps in the current CRDS context are placeholders, as they require calibrations that can only be taken in flight; for this reason, the absolute flux values seen here should not be taken literally.

### Simulated data <a id='sims'></a>

We will be using simulated NIRSpec exposures generated by the ESA Instrument Performance Simulator (IPS), using an input scene consisting of a single point source within the IFU aperture described by an emission-line galaxy template spectrum. The observation consists of four dithered exposures taken using the first four positions in the IFU cycling pattern. The instrument was configured with the G140H+F100LP grating/filter combination. The dithers are numbered for convenience in these example file names according to the ordering of the positions in the pattern. These file names are not indicative of actual data product file names you will see in the archive.

There are a number of caveats to be aware of regarding these simulated data. 1) The IPS does not include a full treatment of all of the effects corrected by the stage 2 pipeline, particularly some of the throughput components. As with the above caveat regarding reference files, the simulations shown here should not be used for any analyses of flux information. 2) The simulated PSF is truncated in order to save on computation time, which results in an artifical drop in apparent count rate in the PSF wings. 3) Spacecraft pointing-related information has to be added by-hand to the headers before the simulated data can be processed by the pipeline. These keywords are used by the stage 3 pipeline when combining exposures, in order to align the spectral traces. Because this has to be a manual process and may be subject to small errors, the quality of the combined products here should not be taken as indicative of actual in-flight performance.

## Imports <a id='imports'></a>

Import packages necessary for this notebook

In [ ]:
import numpy as np

import glob

#Modify the path to a directory on your machine
import os
os.environ["CRDS_PATH"] = "/Users/cpacifici/crds_cache_152/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

import zipfile
import urllib.request

import json

from astropy.io import fits
from astropy.utils.data import download_file
import astropy.units as u
from astropy import wcs
from astropy.wcs import WCS
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch, AsinhStretch

Set up matplotlib for plotting

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline

# Use this version (outside of Jupyter Lab) if you want interactive plots
#%matplotlib notebook

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

plt.rcParams.update({'font.size': 18})

Import JWST pipeline modules

In [ ]:
# The calwebb_spec and spec3 pipelines
from jwst.pipeline import Spec2Pipeline
from jwst.pipeline import Spec3Pipeline

# individual steps
from jwst.assign_wcs import AssignWcsStep
from jwst.assign_wcs import nirspec
from jwst.background import BackgroundStep
from jwst.imprint import ImprintStep
from jwst.msaflagopen import MSAFlagOpenStep
from jwst.extract_2d import Extract2dStep
from jwst.srctype import SourceTypeStep
from jwst.wavecorr import WavecorrStep
from jwst.flatfield import FlatFieldStep
from jwst.pathloss import PathLossStep
from jwst.photom import PhotomStep
from jwst.cube_build import CubeBuildStep
from jwst.extract_1d import Extract1dStep

# data models
from jwst import datamodels

## Define convenience functions and parameters <a id='func'></a>

In [ ]:
# All files created by the steps in this notebook have been pre-computed and cached in the following directory
# if you want to actually run everything offline, then comment out this line and specify a desired local directory
#output_dir = '/home/shared/preloaded-fits/nirspec-files/'
output_dir = './nirspec_files/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
def show_image(data_2d, vmin, vmax, xsize=15, ysize=15, title=None, aspect=1, scale='log', units='MJy/sr'):
    """Function to generate a 2D, log-scaled image of the data
    
    Parameters
    ----------
    data_2d : numpy.ndarray
        2D image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    title : str
        String to use for the plot title
        
    scale : str
        Specify scaling of the image. Can be 'log' or 'linear'
        
    units : str
        Units of the data. Used for the annotation in the
        color bar
    """
    if scale == 'log':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LogStretch())
    elif scale == 'linear':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LinearStretch())
    elif scale == 'Asinh':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=AsinhStretch())
    fig = plt.figure(figsize=(xsize, ysize))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm, aspect=aspect, cmap='gist_earth')

    fig.colorbar(im, label=units)
    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    if title:
        plt.title(title)

## Pipeline processing flag <a id='flag'></a>

The pipeline and individual steps take too long to run in real time for this demo, so all products shown here have been pre-computed, and the actual pipeline calls will be skipped.  Change the following flag to True if you want to run everything offline.

In [ ]:
runflag = True

## Download Data <a id='data'></a>

The input simulated data and all pipeline products are available on Box.  If you want to run the pipeline offline, first download the zip file and unpack in the following cells; just comment out the first line in each.

In [ ]:
#%%script false --no-raise-error

# set the Box link and file name
ziplink = 'https://stsci.box.com/shared/static/c8uu9cn0dldkjv86z5rz16rszb39gjzi.zip'
zipfilename = 'nirspec_ifudata.zip'
if not os.path.isfile(os.path.join(output_dir, zipfilename)):
    print('Downloading {}...'.format(zipfilename))
    demo_file = download_file(ziplink, cache=True)
    # Make a symbolic link using a local name for convenience
    os.symlink(demo_file, os.path.join(output_dir, zipfilename))
else:
    print('{} already exists, skipping download...'.format(zipfilename))

In [ ]:
#%%script false --no-raise-error

# unzip
zf = zipfile.ZipFile(output_dir+'nirspec_ifudata.zip', 'r')
zf.extractall(output_dir)

## Input simulations <a id='inputs'></a>

We will be using simulated NIRSpec exposures of a point source observed with the IFU.  Because the simulator generates count rate maps, equivalent to level 2a data products, we have to skip stage 1 of the pipeline and instead start the processing with the calwebb_spec2 pipeline.
First, let's take a look at a few of the level 2a images to get familiarized with the inputs.  The observation consists of 4 dithered exposures taken with a 4-point dither pattern.

In [ ]:
# get the data model of dither position 1:
ratefile1 = output_dir+'nirspec_ifusim_d1_nrs1_rate.fits' # for the NRS1 detector
dither1 = datamodels.open(ratefile1)
ratefile2 = output_dir+'nirspec_ifusim_d1_nrs2_rate.fits' # for the NRS2 detector
dither2 = datamodels.open(ratefile2)

# get the pixel data (the SCI extension of the fits file)
ratesci1 = dither1.data
ratesci2 = dither2.data

# display the images
show_image(ratesci1, 0, 5.e2, units='DN/s')
show_image(ratesci2, 0, 5.e2, units='DN/s')

## Level 2 association files <a id='lvl2asn'></a>

The automated pipeline uses association files to specify inputs into the calwebb_spec2 pipeline. The primary use for these is to specify multiple exposures to be used as background for subtraction from a given science exposure. In this demo, we're working with a dither set that is optimized for extended sources, and not used for background subtraction, so we will only use the association file to specify the full set of exposures to be processed one-by-one.

In [ ]:
# show the contents of the association file
asn_file = output_dir+"l2_asn.json"
with open(asn_file) as f_obj:
    asn_data = json.load(f_obj)
asn_data

## Run the calwebb_spec2 pipeline <a id='runspec2'></a>

The following cell demonstrates how to run the calwebb_spec2 pipeline in full, using the above association file as input. Since this takes several hours to run in real time, the cell has been disabled for the webinar. In order to run it offline, simply delete or comment out the top line.

In [ ]:
if (runflag == True):
    spec2 = Spec2Pipeline()
    spec2.save_results = True
    spec2.output_dir = output_dir
    # skip the flat field correction, since the simulations do not include a full treatment of the throughput
    spec2.flat_field.skip = True
    result = spec2(asn_file)

In [ ]:
# take a look at the results - open the level 2b files

callist = [f for f in glob.glob(output_dir+"*_cal.fits")]
callist.sort()
for calfile in callist:
    # the unrectified 2D calibrated image
    print(calfile)
    cal = datamodels.MultiSpecModel(calfile) # this contains the calibrated unrectified 2D spectrum
    calsci = cal.data
    root = calfile[: -9]
    
    # the exposure-level cube
    hdu = fits.open(root+'_s3d.fits')
    cube = hdu[1].data
    wcs = WCS(hdu[1].header)
    hdu.close()
    
    # 1D spectrum extracted from the exposure-level cube
    x1d = datamodels.MultiSlitModel(root+'_x1d.fits')  # this contains the aperture-extracted 1D spectrum
 
    # get wavelength & flux from the x1d data model
    x1dwave = x1d.spec[0].spec_table['WAVELENGTH']
    x1dflux = x1d.spec[0].spec_table['FLUX']
    
    # plot the 2D image   
    show_image(calsci, 0, 1.5e2)

    # plot a slice from the cube
    # get slice wavelength values from the WCS transform for plot labels
    nslice = 1000
    sky, wave = wcs.pixel_to_world(0,0,nslice)
    title = np.around(wave.to(u.um),3)
    ax = plt.subplot(1, 1, 1, projection=wcs, slices=('x', 'y', nslice))
    norm=ImageNormalize(cube[nslice, :, :], vmin=0, vmax=5.e2, stretch=LogStretch())
    ax.imshow(cube[nslice, :, :], norm=norm, origin='lower', cmap='gist_earth')
    ax.set_xlabel('RA')
    ax.set_ylabel('DEC')
    ax.grid(color='white', ls='solid')
    ax.set_title(title)
        
    fig = plt.figure(figsize=(19, 8))
    plt.plot(x1dwave, x1dflux)
    plt.xlabel('wavelength (microns)')
    plt.ylabel('flux (Jy)')
    plt.show()

## Run individual steps of the spec2 pipeline <a id='runspec2steps'></a>

Now let's try running individual pipeline steps on a single exposure and single detector, and take a look at their output to give a flavor for what they are doing to the data.

### assign_wcs <a id='awcs'></a>

This is a complex step that generates tranfsorms between various instrument optical planes and the sky, using the NIRSpec instrument model with multiple reference files.

In [ ]:
filename = output_dir+'nirspec_ifusim_d1_nrs1_rate.fits'  # an input level 2a file
root = filename[:-9]  # for later matching
print(root)

if (runflag == True):
    step = AssignWcsStep()
    step.save_results = True
    step.output_dir = output_dir
    result = step(filename)

Note the verbose output prints the values of various instrument parameters that are relevant for the construction of the instrument transforms, such as the grating wheel tilt and the slit (or slits, in the case of MOS) used for the exposure.

In [ ]:
# The output file has the suffix _assignwcsstep appended:
awcs_output_file = root+'assignwcsstep.fits'
print(awcs_output_file)

# load the output into a data model container
awcs = datamodels.open(awcs_output_file)
awcs

The actual pixel data are not changed as a result of this step.  Instead, a WCS object is created in the asdf extension of the output file that includes all the various optical transforms and component descriptions encompassed by the instrument model.

In [ ]:
# get the WCS information populated by the algorithms of the assign_wcs step
wcsobj = awcs.meta.wcs

In [ ]:
# list the frames of reference available for transformation
wcsobj.available_frames

In [ ]:
# examples of the units for some of these frames
print(wcsobj.detector.unit) # xy pixel indices
print(wcsobj.slit_frame.unit)  # relative xy position in the slit aperture
print(wcsobj.msa_frame.unit)  # absolute xy position in the slit aperture
print(wcsobj.world.unit)  # RA, Dec sky position

These correspond to image and pupil planes in the instrument, including the detector, the grating wheel, the slit aperture, and various sky planes.  One can transform between any two of these using the appropriate transformation. Let's try an example of that to plot the wavelength of an emission line on the 2D spectrum.

In [ ]:
# specify the wavelength of a feature of interest
wave0 = 1.2518

# pick one of the IFU slices
ifuslice = nirspec.nrs_wcs_set_input(awcs, 4)  # the source is centered in slice 2 for dither 1
y, x = np.mgrid[:awcs.data.shape[0], : awcs.data.shape[1]]
ra, dec, wave = ifuslice(x, y)
world2det = ifuslice.get_transform('world', 'detector')

# get bounding box of the slice 2d trace
xstart = int(np.ceil(ifuslice.bounding_box[0][0]))
xstop = int(np.ceil(ifuslice.bounding_box[0][1]))
ystart = int(np.ceil(ifuslice.bounding_box[1][0]))
ystop = int(np.ceil(ifuslice.bounding_box[1][1]))
imslice = awcs.data[ystart:ystop, xstart:xstop]
waveslice = wave[ystart:ystop, xstart:xstop]

# find the trace of the feature wavelength in 2D space
xm = np.arange(imslice.shape[1])
ym = np.arange(imslice.shape[0])
xm_wave = np.zeros(waveslice.shape[0])
for i in range(len(ym)):
    col = waveslice[i,:]

    if not np.all(np.isnan(col)):
        xm_wave[i] = np.interp(wave0, col[np.isfinite(col)], xm[np.isfinite(col)], left=0, right=0)

show_image(imslice, 0., 2.e2, xsize=15, ysize=8, aspect=1., units='DN/s')
plt.plot(xm_wave[xm_wave != 0], ym[xm_wave != 0], color='white', alpha=1, linewidth=2)
plt.xlim(900,1250)
plt.show()

### background <a id='background'></a>

If there are associated exposures to be used as backgrounds (as in a nod set), this step will average them (if more than one), and then directly subtract from the exposure being processed.  In this demo, the example data are part of a cycling dither pattern, which is meant for improving the sampling of extended sources, and the dither separations are not large enough to enable image-to-image subtraction without source overlap.  Thus, we will skip running this step (note, the full pipeline automatically skips it if there are no background exposures listed in the association file).

### imprint <a id='imprint'></a>

The MSA shutters are not perfectly opaque; a small amount of light from the sky leaks through the edges of each shutter.  In addition, a very small number of shutters are stuck open and hence always open to sky.  This results in a dispersed "leakage" signal (also referred to as "imprint") that projects onto the same area of the detectors that contain light from the IFU aperture.  Dedicated leakage exposures, taken at the same position as the science exposure with the IFU aperture covered and the MSA all closed, can be obtained to remove this signal.  The imprint step takes the associated leakage exposures and subtracts them pixel-by-pixel from the science exposure.

The simulated exposures we are using for this demo do not contain leakage signal, so we will skip running this step (note, the full pipeline automatically skips it if there are no leakage exposures listed in the association file).

### sourcetype <a id='srctype'></a>

This step sets the source type (point or extended) based on information passed from PPS (given in the SRCTYAPT header keyword); if no flag was set, by default the type is set to extended for IFU data.  This affects subsequent processing steps, which do different things for each type.

In [ ]:
filename = root+'assignwcsstep.fits'  # the output from the previous step

if (runflag == True):
    step = SourceTypeStep()
    step.save_results = True
    step.output_dir = output_dir
    result = step(filename)

keyword SRCTYPE should be set to "POINT" in this case, because PPS database value SRCTYAPT is "POINT"

In [ ]:
# The output file has the suffix _sourcetypestep appended:
stype_output_file = root+'sourcetypestep.fits'

# load the output into a data model container
stype = datamodels.open(stype_output_file)

# what is the SRCTYPE keyword value?
stype.find_fits_keyword('SRCTYPE')
print('SRCTYPE=', stype.meta.target.source_type)

### flat_field <a id='flat'></a>

A flat field correction is applied to each pixel in the 2D spectrum.  The NIRSpec "flat" comprises three components: D flat, which includes the pixel-to-pixel detector response; S flat, which includes the throughput of the spectrograph; F flat, which folds together the throughput of the filter, FORE optics, and OTE.

In [ ]:
filename = root+'sourcetypestep.fits'  # the output from the previous step

if (runflag == True):
    step = FlatFieldStep()
    step.save_interpolated_flat = True  # this will save the on-the-fly flat field correction values as an image
    step.save_results = True
    step.output_dir = output_dir
    result = step(filename)

In [ ]:
# The output file has the suffix _flatfieldstep appended:
flat_output_file = root+'flatfieldstep.fits'

# The optional saved flat correction image has the suffix _interpolatedflat appended:
intflat_output_file = root+'interpolatedflat.fits'

# load the output into a data model container
flat = datamodels.open(flat_output_file)
intflat = datamodels.open(intflat_output_file)

In [ ]:
# plot the flat correction image - this is divided into the science exposure to apply the corrections per pixel
show_image(intflat.data, 0.3, 1., scale='Asinh', units='DN/s')

In [ ]:
# plot the corrected science exposure
# note the simulation does not include all of the flat components, so applying the full correction in this case produces incorrect results
show_image(flat.data, 0., 8.e1, scale='Asinh', units='DN/s')

### pathloss <a id='pathloss'></a>

The pathloss correction scales the 2D spectrum as a function of wavelength to account for diffraction losses incurred by the grating wheel. These will not be measured until flight, so the current reference file is only a placeholder and the correction factors are unity.

In [ ]:
filename = root+'flatfieldstep.fits'  # the output from the previous step

if (runflag == True):
    step = PathLossStep()
    step.save_results = True
    step.output_dir = output_dir
    result = step(filename)

In [ ]:
# The output file has the suffix _pathlossstep appended:
ploss_output_file = root+'pathlossstep.fits'

# load the output into a data model container
ploss = datamodels.open(ploss_output_file)

In [ ]:
# plot the corrected science exposure
show_image(ploss.data, 0., 8.e1, scale='Asinh', units='DN/s')

### photom <a id='photom'></a>

The photom step applies a scalar conversion factor to convert to physical units.  Once on-orbit observations of spectrophotometric standards are obtained, this may also include a wavelength-dependent vector, if necessary, to account for any small discrepancies found in the throughput corrections.

In [ ]:
filename = root+'pathlossstep.fits'  # the output from the previous step

if (runflag == True):
    step = PhotomStep()
    step.save_results = True
    step.output_dir = output_dir
    result = step(filename)

In [ ]:
# The output file has the suffix _photomstep appended:
phot_output_file = root+'photomstep.fits'

# load the output into a data model container
phot = datamodels.open(phot_output_file)

In [ ]:
# what are the flux calibration-related keywords?
phot.find_fits_keyword('PHOTUJA2')
print('PHOTMJSR=', phot.meta.photometry.conversion_megajanskys)
# note that despite the keyword name, when SRCTYPE=POINT, the units are actually MJy per pixel
# the pre-launch reference file has a placeholder value, so the output fluxes will appear unphysically large
print('PHOTUJA2=', phot.meta.photometry.conversion_microjanskys)
# this is only applicable for extended sources

### cube_build <a id='cubebuild'></a>

The fully calibrated 2D spectrum is resampled onto a rectified cube in sky and wavelength coordinates. This is intended for quick-look purposes only, as all of the calibrated unrectified exposures will be resampled and combined onto a single cube during the level 3 processing.

In [ ]:
filename = root+'photomstep.fits'  # the output from the previous step

if (runflag == True):
    step = CubeBuildStep()
    step.save_results = True
    step.output_dir = output_dir
    result = step(filename)

In [ ]:
# The output file has the suffix _photomstep appended:
cube_output_file = root+'photomstep_g140h-f100lp_s3d.fits'
print(cube_output_file)

# read in the output fits file
hdu = fits.open(cube_output_file)
# get the exposure-level cube
cube = hdu[1].data
# get the WCS info
wcs = WCS(hdu[1].header)
cdelt = hdu[1].header['CDELT1']*3600.
hdu.close()

In [ ]:
# show a slice

nslice = 1750 # which wavelength slice of the cube to use for display & fitting
# get slice wavelength values from the WCS transform for plot labels
nslice = 1750
sky, wave = wcs.pixel_to_world(0,0,nslice)
title = np.around(wave.to(u.um),3)
show_image(cube[nslice,:,:], 0., 1.e3, units='MJy')
plt.xlabel('RA')
plt.ylabel('DEC')
plt.grid(color='white', ls='solid')
plt.title(title)

### extract_1d <a id='extract1d'></a>

A 1D spectrum is extracted from the cube, using a circular aperture with radius specified from a reference file.  An aperture correction is also applied to account for flux outside of the extraction aperture. As with the level 2b cube, this is intended for quick-look purposes only.  An independent extraction of the combined cube will be done during level 3 processing.

In [ ]:
filename = root+'photomstep_g140h-f100lp_s3d.fits'  # the output from the previous step

if (runflag == True):
    step = Extract1dStep()
    step.save_results = True
    step.output_dir = output_dir
    result = step(filename)

In [ ]:
# The output file has the suffix _extract1dstep appended:
x1d_output_file = root+'photomstep_g140h-f100lp_extract1dstep.fits'

# load the output into a data model container
x1d = datamodels.open(x1d_output_file)

In [ ]:
# plot the spectrum
x1dwave = x1d.spec[0].spec_table.WAVELENGTH
x1dflux = x1d.spec[0].spec_table.FLUX

fig = plt.figure(figsize=(19, 8))
plt.plot(x1dwave, x1dflux)
plt.xlabel('wavelength (microns)')
plt.ylabel('flux (Jy)')
plt.show()

# note that the flux level appears artificially inflated because of the dummy flux conversion factor applied in the photom step

## Run the calwebb_spec3 pipeline <a id='runspec3'></a>

For an observation that contains multiple exposures of the same source, such as the dither set in this example, this pipeline will combine all of the exposures into a single output product.  Both a 3D combined cube and extracted 1D spectrum are generated.  This process also includes an outlier detection step that compares the stack of values at each resampled pixel and flags outliers based on noise threshold parameters; the flagged outliers are not included in the spectral combination.  However, the current implementation of this step has not been properly tuned and tends to flag non-outliers associated with the source PSF, so we will skip it for this demo.  We will also not demonstrate the optional "master background" step, which takes an independent 1D background spectrum to create a 2D background to subtract in the absence of nodded exposures; see the read-the-docs pages for more details.

In [ ]:
# show the contents of the association file
asn_file = output_dir+"l3_asn.json"
with open(asn_file) as f_obj:
    asn_data = json.load(f_obj)
asn_data

run the calwebb_spec3 pipeline using association file of cal.fits files

In [ ]:
if (runflag == True):
    spec3 = Spec3Pipeline()
    spec3.save_results = True
    spec3.output_dir = output_dir
    spec3.outlier_detection.skip = True  # skip this step for now, because the simulations do not include outliers such as CR hits or bad pixels
    result = spec3(asn_file)

In [ ]:
# display level 3 products
# combined 3D cube
hdu = fits.open(output_dir+'nirspec_ifusim_comb_1234_g140h-f100lp_s3d.fits')
cube = hdu[1].data
wcs = WCS(hdu[1].header)
cdelt = hdu[1].header['CDELT1']*3600.
hdu.close()    

# plot slices from the cube
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17,8))

# get slice wavelength values from the WCS transform for plot labels
nslice1 = 1000
nslice2 = 3500
sky, wave = wcs.pixel_to_world(0, 0, nslice1)
title1 = np.around(wave.to(u.um), 3)
sky, wave = wcs.pixel_to_world(0, 0, nslice2)
title2 = np.around(wave.to(u.um), 3)

plt.rcParams.update({'font.size': 16})
ax1 = plt.subplot(1, 2, 1, projection=wcs, slices=('x', 'y', nslice))
norm=ImageNormalize(cube[nslice, :, :], vmin=0, vmax=5.e2, stretch=LogStretch())
slice1 = ax1.imshow(cube[nslice, :, :], norm=norm, origin='lower', cmap='gist_earth')
ax1.set_xlabel('RA')
ax1.set_ylabel('DEC')
ax1.grid(color='white', ls='solid')
ax1.set_title(title1)

ax2 = plt.subplot(1, 2, 2, projection=wcs, slices=('x', 'y', nslice))
norm=ImageNormalize(cube[nslice, :, :], vmin=0, vmax=5.e2, stretch=LogStretch())
slice2 = ax2.imshow(cube[nslice, :, :], norm=norm, origin='lower', cmap='gist_earth')
ax2.set_xlabel('RA')
ax2.set_ylabel('DEC')
ax2.grid(color='white', ls='solid')
ax2.set_title(title2)

In [ ]:
# combined 1D extracted spectrum
x1d3 = datamodels.open(output_dir+'nirspec_ifusim_comb_1234_g140h-f100lp_x1d.fits')

# get wavelength & flux
x1d3wave = x1d3.spec[0].spec_table.WAVELENGTH
x1d3flux = x1d3.spec[0].spec_table.FLUX

# plot
fig = plt.figure(figsize=(15,9))
plt.plot(x1d3wave,x1d3flux)
plt.xlabel('wavelength (microns)')
plt.ylabel('flux (Jy)')
plt.show()